### Carga de datos

In [3]:
import os
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cv2
import PIL.Image as Image

from skimage import io, color, transform
from skimage.feature import hog

import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report
gray_matrix = np.load('DatosDF/gray_matrix.npy', allow_pickle=True)
hog = np.load('DatosDF/hog.npy', allow_pickle=True)
estados = np.load('DatosDF/estados.npy', allow_pickle=True)
label = np.load('DatosDF/labels.npy', allow_pickle=True)
df = pd.DataFrame({'gray_matrix': gray_matrix,'hog': hog,'estados': estados,'label': label})
dfxusar = df[df['estados'] == 1]
elresto = df[df['estados'] == 0]
dfxusar
X = np.array(dfxusar['hog'].values.tolist())
y = dfxusar['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19011)

## RF Optimo 

**Parametros:**
- 'max_depth': None
- 'min_samples_leaf': 4
- 'min_samples_split': 14

In [8]:
# Crear el modelo
rf_classifier = RandomForestClassifier(max_depth=None, min_samples_leaf=4,min_samples_split=14)

# Entrenar el modelo
rf_classifier.fit(X_train, y_train)

y_pred_e= rf_classifier.predict(X_train)
accuracy = accuracy_score(y_train, y_pred_e)
print(f'Precisión del modelo: {accuracy}')

print(classification_report(y_train, y_pred_e))

# Realizar predicciones en el conjunto de prueba
y_pred = rf_classifier.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {accuracy}')

print(classification_report(y_test, y_pred))

Precisión del modelo: 0.9964750542299349
              precision    recall  f1-score   support

       angry       1.00      0.99      1.00       462
     disgust       1.00      1.00      1.00       435
        fear       1.00      0.99      1.00       579
       happy       0.98      1.00      0.99       650
     neutral       1.00      1.00      1.00       622
         sad       1.00      0.99      1.00       467
    surprise       1.00      1.00      1.00       473

    accuracy                           1.00      3688
   macro avg       1.00      1.00      1.00      3688
weighted avg       1.00      1.00      1.00      3688

Precisión del modelo: 0.7941495124593716
              precision    recall  f1-score   support

       angry       0.82      0.67      0.74       101
     disgust       0.78      0.47      0.59       112
        fear       0.75      0.71      0.73       156
       happy       0.83      1.00      0.91       149
     neutral       0.78      0.90      0.83       

## MLP Optimo

**Parametros**
- 'batch_size': 512
- 'learning_rate_init': 0.1
- 'max_iter': 1000

In [7]:
mlp = MLPClassifier(hidden_layer_sizes=(100,),
                    activation='logistic',
                    solver='sgd',
                    alpha=0,
                    batch_size = 512,
                    learning_rate='constant',
                    learning_rate_init=0.1,
                    max_iter=1000,
                    tol=1e-6,
                    verbose=True,
                    momentum=0,
                    n_iter_no_change=25,
                    random_state=42)
mlp.fit(X_train, y_train)
y_pred_e= mlp.predict(X_train)
accuracy = accuracy_score(y_train, y_pred_e)
print(f'Precisión del modelo: {accuracy}')

print(classification_report(y_train, y_pred_e))
# Realizar predicciones en el conjunto de prueba
y_pred = mlp.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {accuracy}')

print(classification_report(y_test, y_pred))

Iteration 1, loss = 1.95300331
Iteration 2, loss = 1.92639804
Iteration 3, loss = 1.91688857
Iteration 4, loss = 1.91065132
Iteration 5, loss = 1.90098490
Iteration 6, loss = 1.89152238
Iteration 7, loss = 1.88172526
Iteration 8, loss = 1.87285940
Iteration 9, loss = 1.86167621
Iteration 10, loss = 1.85221062
Iteration 11, loss = 1.84107689
Iteration 12, loss = 1.83039780
Iteration 13, loss = 1.81752354
Iteration 14, loss = 1.80557674
Iteration 15, loss = 1.79232196
Iteration 16, loss = 1.77672227
Iteration 17, loss = 1.76391182
Iteration 18, loss = 1.74705563
Iteration 19, loss = 1.73236709
Iteration 20, loss = 1.71548377
Iteration 21, loss = 1.69714331
Iteration 22, loss = 1.67902846
Iteration 23, loss = 1.66204622
Iteration 24, loss = 1.64401322
Iteration 25, loss = 1.62183226
Iteration 26, loss = 1.60391507
Iteration 27, loss = 1.58275507
Iteration 28, loss = 1.56246515
Iteration 29, loss = 1.54260486
Iteration 30, loss = 1.52184046
Iteration 31, loss = 1.50064865
Iteration 32, los

C:\Users\gonza\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


## KNN Optimo

**Parametros:**
- 'n_neighbors': 3 
- 'p': 2


In [5]:
knn = KNeighborsClassifier(n_neighbors=3,p=2)  
knn.fit(X_train, y_train)
y_pred_e= knn.predict(X_train)
accuracy = accuracy_score(y_train, y_pred_e)
print(f'Precisión del modelo (train): {round(accuracy,5)}')

print(classification_report(y_train, y_pred_e))

y_pred = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo (test): {round(accuracy,5)}')

print(classification_report(y_test, y_pred))

Precisión del modelo (train): 0.93818
              precision    recall  f1-score   support

       angry       0.91      0.97      0.94       462
     disgust       0.92      0.83      0.87       435
        fear       0.92      0.92      0.92       579
       happy       0.96      1.00      0.98       650
     neutral       0.93      0.97      0.95       622
         sad       0.98      0.84      0.91       467
    surprise       0.94      0.99      0.97       473

    accuracy                           0.94      3688
   macro avg       0.94      0.93      0.93      3688
weighted avg       0.94      0.94      0.94      3688

Precisión del modelo (test): 0.82124
              precision    recall  f1-score   support

       angry       0.70      0.82      0.75       101
     disgust       0.69      0.50      0.58       112
        fear       0.78      0.84      0.81       156
       happy       0.84      0.99      0.91       149
     neutral       0.86      0.96      0.91       153
   